# Insertion de nouveaux points

In [1]:
import os
import glob


directory = "results/assets"
notebook_path = os.path.abspath("Notebook.ipynb")
chemin = os.path.join(os.path.dirname(notebook_path), directory)
paths = glob.glob(chemin+'/*')

print("done")

done


In [2]:
from insertion_data import *

client = pymongo.MongoClient('localhost',27017)
collection = client.fcd.assets

print('done')

done


In [3]:
insert_data(paths, "assets", collection)


reading...
reading done
insertion...
insertion done
stay points research...
3436 stay points detected

done


In [4]:
# ajout index géo spatial
add_2dsphere_index(collection, "location.geo")

index response: location.geo_2dsphere
